In [8]:
# -*- coding: utf-8 -*-
import os
import numpy as np
import cv2
import dlib
import time
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Load model

In [9]:
detector = dlib.get_frontal_face_detector()
p = "shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor (p)
face_rec_model_path = 'dlib_face_recognition_resnet_model_v1.dat'
facerec = dlib.face_recognition_model_v1(face_rec_model_path)

## get facial imformation 

If an image contains no face, it will not be included in the pkl file

In [10]:
result = []
def get_faces(file):
    image = cv2.imdecode(np.fromfile(file,dtype=np.uint8),-1)
    image = cv2. cvtColor (image, cv2.COLOR_BGR2RGB)
    gray = cv2. cvtColor (image, cv2.COLOR_RGB2GRAY)

    rects = detector (gray , 0)
    if len(rects) >=0:
        for rect in rects :
            shape = predictor (gray , rect )
            face_chip = dlib.get_face_chip(image, shape)
            face_descriptor = facerec.compute_face_descriptor(face_chip)
            item = [file, [rect.left(), rect.top(), rect.width(), rect.height()], list(face_descriptor), 0, 0]
            result.append(item)
        
def recu_dir(org_dir):
    for file in os.listdir(org_dir):
        if os.path.isdir(os.path.join(org_dir,file)):
            abspath = os.path.join(org_dir,file)
            recu_dir(os.path.join(org_dir, file))
        else:
            if file[-4:] == '.jpg':
                get_faces(os.path.join(org_dir, file))


In [11]:
recu_dir('D://Data')

AssertionError: 5 columns passed, passed data had 3 columns

In [14]:
data = pd.DataFrame(result, columns=['file', 'rect', 'face_descriptor'])
data.head()

,file,rect,face_descriptor
0,D://Data\video\8.29张江人工智能岛\29号\摄像迟\BPAV\CLPR\4...,"[925, 365, 73, 73]","[-0.10779663920402527, 0.10153406113386154, 0...."
1,D://Data\video\8.29张江人工智能岛\29号\摄像迟\BPAV\CLPR\4...,"[813, 362, 87, 87]","[-0.060471706092357635, 0.036922015249729156, ..."
2,D://Data\video\8.29张江人工智能岛\29号\摄像迟\BPAV\CLPR\4...,"[369, 231, 126, 125]","[-0.09775551408529282, 0.07954566180706024, 0...."
3,D://Data\video\8.29张江人工智能岛\29号\摄像迟\BPAV\CLPR\4...,"[353, 238, 105, 105]","[-0.09728292375802994, 0.04980679973959923, 0...."
4,D://Data\video\8.29张江人工智能岛\29号\摄像迟\BPAV\CLPR\4...,"[839, 176, 126, 125]","[-0.003757824655622244, 0.09656134992837906, 0..."


In [15]:
data.to_pickle('faces_full.pkl')

In [16]:
data.shape

(28304, 3)